<a href="https://colab.research.google.com/github/mantoan-thi/Wire-harness-analysis/blob/main/Comparando_chicotes_Yazaki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlsxwriter

     |████████████████████████████████| 148 kB 5.1 MB/s 


In [277]:
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from datetime import timedelta

In [122]:
# Carregando o Datastet
df_circuitos = pd.read_excel('/content/Consulta BOM v15.xlsm',sheet_name='CIRCUITO')
df_componentes = pd.read_excel('/content/Consulta BOM v15.xlsm',sheet_name='COMPONENTE')
df_tubos = pd.read_excel('/content/Consulta BOM v15.xlsm',sheet_name='TUBO')
df_sumarizado = pd.read_excel('/content/Consulta BOM v15.xlsm',sheet_name='SUMARIZADO')

# Análise de circuitos

In [180]:
# Selecionando somentes colunas importantes
if 'OT_DISCRIMINADA' in df_circuitos.columns:
  df_circuitos = df_circuitos[['PROD_TERM', 'OT', 'OT_DISCRIMINADA', 'CIRC_ESQ','CIR_DIR', 'TIPO', 
                              'BITOLA', 'COR', 'COMP', 'COD_OP_01_ESQ', 'TERMINAL_ESQ','ACC_ESQ_1',
                              'COD_OP_01_DIR','TERMINAL_DIR','ACC_DIR_1','SOLDA']].copy()

  # Obtenha os nomes dos índices para os quais a coluna 'OT_DISCRIMINADA' tem o valor SIM

  index_ot_discriminada = df_circuitos[df_circuitos['OT_DISCRIMINADA']=='SIM'].index
  if index_ot_discriminada.shape[0] > 0: # Excluindo esses índices de linha de dataFrame
    df_circuitos.drop(index_ot_discriminada, inplace=True)
    df_circuitos.drop(columns=['OT_DISCRIMINADA'],axis=1, inplace=True)
  df_circuitos.fillna(0,inplace=True)

df_circuitos['COD_OP_01_ESQ'] = df_circuitos['COD_OP_01_ESQ'].apply(lambda x: 0 if x != 'U' else 'U')
df_circuitos['COD_OP_01_DIR'] = df_circuitos['COD_OP_01_DIR'].apply(lambda x: 0 if x != 'U' else 'U')
df_circuitos.head()
# Quantidade de circuitos por part number
lista_pn = []
pn_anterior = 0
for pn in df_circuitos['PROD_TERM']:
  if pn_anterior != pn:
    pn_anterior = pn
    qtde = df_circuitos[df_circuitos['PROD_TERM']==pn].shape[0]
    lista_pn.append([pn,qtde])
lista_pn =pd.DataFrame(lista_pn,columns=['Part_number','Qtde'])
lista_pn.sort_values(by=['Qtde'],ascending=False,inplace=True)
lista_pn.reset_index(inplace=True,drop=True)


qtde_pn = lista_pn.shape[0]
menor_pn = lista_pn['Part_number'][-1:].values[0]
maior_pn = lista_pn['Part_number'][0]
Describ = lista_pn.describe()


# Calcular a quantidade de comparações
contador1 =0
contador2 = 0
lista_de_comparacao = []
Total = 0
while contador1 < lista_pn['Part_number'].shape[0]:
  contador2=contador1+1
  while contador2 < lista_pn['Part_number'].shape[0]:
    lista_de_comparacao.append(lista_pn['Part_number'][contador1]+' vs '+lista_pn['Part_number'][contador2])
    contador2+=1
    Total+=1
  contador1+=1
  contador2 = 0
lista_de_comparacao = pd.DataFrame(lista_de_comparacao,columns=['Part Numbers'])

In [278]:
def analisar(list_comp,tol):
  contar1 = 0
  contar2 = 0
  list_comp['Delta'] = 0
  while contar1 < list_comp.shape[0]:
    contar2 = contar1+1
    while contar2 < list_comp.shape[0]:
      if list_comp['PROD_TERM'].iloc[contar1]!=list_comp['PROD_TERM'].iloc[contar2]:
        # Tipo+bit+cor
        tipoA = list_comp['TIPO'].iloc[contar1]+list_comp['BITOLA'].iloc[contar1]+list_comp['COR'].iloc[contar1]
        tipoB = list_comp['TIPO'].iloc[contar2]+list_comp['BITOLA'].iloc[contar2]+list_comp['COR'].iloc[contar2]
        TAE = str(list_comp['COD_OP_01_ESQ'].iloc[contar1])+str(list_comp['TERMINAL_ESQ'].iloc[contar1])+str(list_comp['ACC_ESQ_1'].iloc[contar1])
        TAD = str(list_comp['COD_OP_01_DIR'].iloc[contar1])+str(list_comp['TERMINAL_DIR'].iloc[contar1])+str(list_comp['ACC_DIR_1'].iloc[contar1])
        TBE = str(list_comp['COD_OP_01_ESQ'].iloc[contar2])+str(list_comp['TERMINAL_ESQ'].iloc[contar2])+str(list_comp['ACC_ESQ_1'].iloc[contar2])
        TBD = str(list_comp['COD_OP_01_DIR'].iloc[contar2])+str(list_comp['TERMINAL_DIR'].iloc[contar2])+str(list_comp['ACC_DIR_1'].iloc[contar2])
        COMPA = int(list_comp['COMP'].iloc[contar1])
        COMPB = int(list_comp['COMP'].iloc[contar2])
        dif = abs(COMPA-COMPB)
        JTA = str(list_comp['SOLDA'].iloc[contar1])[0]
        JTB = str(list_comp['SOLDA'].iloc[contar2])[0]
        if tipoA == tipoB:                            # Comparando Tipo,Bit e cor
          if COMPA == COMPB or dif < tol:              # Checando dimensional
            if TAE==TBE and TAD==TBD:                  # Terminais
              list_comp['Delta'].iloc[contar1] ='Comum' 
              list_comp['Delta'].iloc[contar2] ='Comum'
            if TAE==TBD and TAD==TBE:                  # Terminais
              list_comp['Delta'].iloc[contar1] ='Comum' 
              list_comp['Delta'].iloc[contar2] ='Comum'

      contar2 +=1
    contar1+=1
    contar2 = 0
    list_comp['Delta'].loc[list_comp['Delta']==0]='DIF'
    lista_analisada = list_comp.sort_values(by=['OT'],ascending=True,inplace=True)
  return list_comp

In [182]:
# Part number que será o 'PARA'
partA = lista_pn['Part_number'][7]
partA[5:10]

'F5380'

In [306]:
contador = 0
lista_de_pn_comparados = []
for pn in lista_pn['Part_number']:
  if pn != partA:
    list_comp = df_circuitos[(df_circuitos['PROD_TERM']==pn) | (df_circuitos['PROD_TERM']==partA)]
    start = timer()
    nova_lista = analisar(list_comp,50)
    nova_lista['Total'] = 1
    nova_lista['Delta'].loc[(nova_lista['Delta']=='DIF') & (nova_lista['PROD_TERM']==partA)]='Adicionar'
    nova_lista['Delta'].loc[(nova_lista['Delta']=='DIF') & (nova_lista['PROD_TERM']==pn)]='Excluir'

    nova_lista1 = nova_lista.copy()
    nova_lista2 = nova_lista.copy()
    nova_lista3 = pd.pivot_table(nova_lista1,index=['OT', 'CIRC_ESQ', 'CIR_DIR', 'TIPO', 'BITOLA', 'COR',
       'COMP', 'COD_OP_01_ESQ', 'TERMINAL_ESQ', 'ACC_ESQ_1', 'COD_OP_01_DIR',
       'TERMINAL_DIR', 'ACC_DIR_1', 'SOLDA','Delta'],values=["Total"],columns=['PROD_TERM'],aggfunc=[np.sum],fill_value=0,)
    
    nova_lista3.columns = [col[2] for col in nova_lista3.columns]
    nova_lista3.reset_index(inplace=True)


   
    resumo = pd.pivot_table(nova_lista2,index=['PROD_TERM','Delta'],values=["Total"],aggfunc=np.sum)

    resumo = pd.DataFrame(resumo)
    resumo.reset_index(inplace=True)
    end = timer()
    tempo_transcorrido = timedelta(seconds=end-start)

    comuns = 0
    adicionar = 0
    Excluir = 0
    contador = 0
    while contador < int(resumo.shape[0])-1:
      if resumo['Delta'][contador] == 'Comum':
        comuns = resumo['Total'][contador]
      if resumo['Delta'][contador] == 'Adicionar':
        adicionar = resumo['Total'][contador]
      if resumo['Delta'][contador] == 'Excluir':
        Excluir = resumo['Total'][contador]
      contador+=1
    perc = '{:.2%}'.format(1-((adicionar+Excluir)/(comuns*2)))

    lista_de_pn_comparados.append([str(tempo_transcorrido)[0:7],pn,partA,comuns,adicionar,Excluir,perc])



lista_de_pn_comparados  = pd.DataFrame(lista_de_pn_comparados,columns=['Tempo','DE','PARA','Comuns','Adicionar','Excluir','Similaridade'])
lista_de_pn_comparados.sort_values(by=['Similaridade'],ascending=False,inplace=True)
# Usando o ExcelWriter, cria um doc .xlsx, usando engine='xlsxwriter'
writer = pd.ExcelWriter('/content/Analise_'+pn[5:10]+' vs '+partA[5:10]+'.xlsx', engine='xlsxwriter')
# Armazena cada df em uma planilha diferente do mesmo arquivo
lista_de_pn_comparados.to_excel(writer, sheet_name='Resumo',index=False)
lista_de_comparacao.to_excel(writer, sheet_name='Lista_comparações')
lista_pn.to_excel(writer, sheet_name='Part_number_Quant')
#nova_lista3.to_excel(writer, sheet_name='Lista_analisada',index=False)

# Fecha o ExcelWriter e gera o arquivo .xlsx
writer.save()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from s

In [305]:
lista_de_pn_comparados.sort_values(by=['Similaridade'],ascending=False).reset_index(drop=True)

,Tempo,DE,PARA,Comuns,Adicionar,Excluir,Similaridade
0,0:00:06,82111F5F30 000,82115F5380 020,127,23,27,80.31%


In [303]:
str(tempo_transcorrido)[0:7]

'0:00:06'

# Trabalhando com tabelas